In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
root = Path('~/non-dropbox-docs/student_data/cs344/23sp').expanduser().resolve()
repos = root / 'repos'
repos.mkdir(parents=True, exist_ok=True)
main_repo = repos / 'main'
main_repo.mkdir(parents=True, exist_ok=True)

In [ ]:
repos

In [ ]:
ghclass_data = pd.read_csv(root / 'portfolio-repo-grades-23sp.csv')
ghclass_data.head()

In [ ]:
import re

student_name_re = re.compile(r'^(.+) \((.+)\)$')

def get_remote_name(student_name_field):
    if pd.isna(student_name_field):
        return None
    student_name, student_id = student_name_re.match(student_name_field).groups()
    return f'{student_id}_{student_name.replace(" ", "_")}'

In [ ]:
ghclass_data['remote_name'] = [get_remote_name(x) for x in ghclass_data['roster_identifier']]

# Initial setup

Clone the template repository.

In [ ]:
template_repo = ghclass_data['starter_code_url'][0]
template_repo = '/'.join(template_repo.rsplit('/', 2)[-2:])
template_repo

In [ ]:
# Clone this repo into the root directory
!git clone git@github.com:{template_repo} {main_repo}

Add all of the student repos as remotes.

In [ ]:
for i in range(len(ghclass_data)):
    student_name = ghclass_data['roster_identifier'].iloc[i]
    if pd.isna(student_name):
        continue
    remote_name = ghclass_data['remote_name'].iloc[i]
    student_repo = ghclass_data['student_repository_url'].iloc[i].replace('https://github.com/', 'git@github.com:')
    print(remote_name)
    !cd {main_repo} && git remote add {remote_name} {student_repo}

Add a worktree for each remote.

In [ ]:
for i in range(len(ghclass_data)):
    remote_name = ghclass_data['remote_name'].iloc[i]
    if pd.isna(remote_name): continue
    print(remote_name)
    #!cd {main_repo} && git worktree add --track -b {remote_name}-main {repos}/{remote_name} {remote_name}/main
    !cd {main_repo} && git worktree add --track -B {remote_name}-main {repos}/{remote_name} {remote_name}/main
    #!cd {main_repo} && git worktree remove {repos}/{remote_name}

Fetch all remotes.

In [ ]:
for i in range(len(ghclass_data)):
    remote_name = ghclass_data['remote_name'].iloc[i]
    if pd.isna(remote_name):
        continue
    print(remote_name)
    !cd {main_repo} && git fetch {remote_name}


In [ ]:
!cd {main_repo} && git gc

The below is for getting Homework 1 stuff.

In [ ]:
import re
url_re = re.compile(r'https://students.cs.calvin.edu/.+(\.zip|\.tgz|\.tar.gz)')

In [ ]:
moodle_assignments = [x for x in Path('../student_data/23sp/23SP CS-344-A-Homework 1-1575205/').glob('**/onlinetext.html')]
for f in moodle_assignments:
    x = f.read_text()
    if 'http' in x:
        print(x)
    if m := url_re.search(x):
        print(m)

Grep those files for students.cs.calvin.edu links.

Download all the links